# Consignes

Intégrer le fichier USvideos.csv qui représente un ensemble de 8000 vidéos Youtube. 

Merger le fichier US_category_id.json pour récupérer le nom des catégories. Il conviendra de bien spécifier l'ID du document.


# Questions 
- 1) Récupérer toutes les vidéos de la chaîne Apple.
- 2) Compter le nombre de catégories différentes
- 3) Si vous ne l'avez pas déjà fait, découper les tags en listes et mettre à jour les tags de chacun des documents avec une requête update.
- 4) Récupérer les vidéos les plus vues.
- 5) Compter le nombre moyen de vues en fonction de la catégorie.
- 6) Récupérer les chaines Youtube avec la plus grande moyenne de likes.

In [1]:
import pandas as pd
import pymongo

In [2]:
client = pymongo.MongoClient(host=['mongodb://172.18.0.5:27017'])
database = client['exercices']
collection = database['youtube']

In [3]:
df_youtube = pd.read_csv("./data/USvideos.csv")
df_youtube.head(2)

,video_id,title,channel_title,category_id,tags,views,likes,dislikes,comment_total,thumbnail_link,date
0,XpVt6Z1Gjjo,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Logan Paul Vlogs,24,logan paul vlog|logan paul|logan|paul|olympics...,4394029,320053,5931,46245,https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg,13.09
1,K4wEI5zhHB0,iPhone X — Introducing iPhone X — Apple,Apple,28,Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lig...,7860119,185853,26679,0,https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg,13.09


## Question 0

### Netoyer les données

In [53]:
df_youtube['date'] = pd.to_datetime(df_youtube['date'], errors='coerce')
df_youtube = df_youtube.dropna(subset=['date'])


df_youtube.rename(columns={'video_id':'_id'}, inplace=True)

df_youtube = df_youtube.drop_duplicates(subset='_id', keep='last')

dataJson =df_youtube.to_dict(orient='records')

import json
with open('./data/US_category_id.json', 'r') as f:
    categ = json.load(f)

for video in dataJson:
    for cat in categ['items']:
        if video['category_id'] == int(cat['id']):
            video['category_name'] = cat['snippet']['title']
            
for video in dataJson:
    video['tags'] = video['tags'].split('|')

In [57]:
dataJson

[{'_id': '5ywKal6-anc',
  'title': 'Gigi Hadid Loses High Heel During Fashion Week, Walks It Off Like a Pro | TMZ',
  'channel_title': 'TMZ',
  'category_id': 24,
  'tags': ['TMZ2016FS11221',
   'TMZ',
   'Hollywood',
   'Celebrity',
   'Entertainment',
   'Famous',
   'Hollywood News',
   'Fame',
   'Entertainment News'],
  'views': 703750,
  'likes': 2921,
  'dislikes': 2196,
  'comment_total': 1042,
  'thumbnail_link': 'https://i.ytimg.com/vi/5ywKal6-anc/default.jpg',
  'date': Timestamp('1970-01-01 00:00:00.000000013'),
  'category_name': 'Entertainment'},
 {'_id': 'GGm0FQ6i74U',
  'title': 'What Hillary Clinton really thinks',
  'channel_title': 'Vox',
  'category_id': 25,
  'tags': ['vox.com',
   'vox',
   'explain',
   'ezra klein',
   'hillary clinton',
   'what happened',
   'donald trump',
   'election 2016',
   'election 2018',
   'bernie sanders',
   'vox conversations',
   'hillary clinton book',
   'hillary clinton interview',
   'hillary clinton book interview',
   'what

### Importer les données

In [58]:
db_exercices = client.exercices
collection_youtube = db_exercices['youtube']

In [59]:
collection_youtube.insert_many(dataJson)

BulkWriteError: batch op errors occurred, full error: {'writeErrors': [{'index': 0, 'code': 11000, 'keyPattern': {'_id': 1}, 'keyValue': {'_id': '5ywKal6-anc'}, 'errmsg': 'E11000 duplicate key error collection: exercices.youtube index: _id_ dup key: { _id: "5ywKal6-anc" }', 'op': {'_id': '5ywKal6-anc', 'title': 'Gigi Hadid Loses High Heel During Fashion Week, Walks It Off Like a Pro | TMZ', 'channel_title': 'TMZ', 'category_id': 24, 'tags': ['TMZ2016FS11221', 'TMZ', 'Hollywood', 'Celebrity', 'Entertainment', 'Famous', 'Hollywood News', 'Fame', 'Entertainment News'], 'views': 703750, 'likes': 2921, 'dislikes': 2196, 'comment_total': 1042, 'thumbnail_link': 'https://i.ytimg.com/vi/5ywKal6-anc/default.jpg', 'date': Timestamp('1970-01-01 00:00:00.000000013'), 'category_name': 'Entertainment'}}], 'writeConcernErrors': [], 'nInserted': 0, 'nUpserted': 0, 'nMatched': 0, 'nModified': 0, 'nRemoved': 0, 'upserted': []}

In [60]:
db_exercices.list_collection_names()

['youtube', 'kickstarter', '{query :{state:"successful"}, out:"sumSuccess"}']

## Question 1  

In [61]:
cur = collection_youtube.find({"channel_title":"Apple"})
list(cur)

[{'_id': 'K4wEI5zhHB0',
  'title': 'iPhone X — Introducing iPhone X — Apple',
  'channel_title': 'Apple',
  'category_id': 28,
  'tags': 'Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lighting|A11 Bionic|augmented reality|emoji|animoji|Face ID|Apple Pay|camera|smartphone',
  'views': 19707391,
  'likes': 381919,
  'dislikes': 69465,
  'comment_total': 0,
  'thumbnail_link': 'https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg',
  'date': datetime.datetime(1970, 1, 1, 0, 0),
  'category_name': 'Science & Technology'},
 {'_id': 'A9k88sMyiJM',
  'title': 'Apple Watch Series 3 + Apple Music — Roll — Apple',
  'channel_title': 'Apple',
  'category_id': 28,
  'tags': 'Apple|airpods|apple watch|apple watch series 3|apple watch cellular|apple watch 4g|apple music|40 million songs on your wrist|apple skateboard ad|apple train station ad|apple roll ad|apple watch skateboard ad|apple watch train station ad|apple watch roll ad|airpods skateboard ad|airpods train station ad|airpods roll ad|apple music skat

## Question 2

In [81]:
cur = collection_youtube.distinct('category_id')
result = 0
for value in cur:
    result = result + 1
print(result)


Il y a 
16
catégories différentes


## Question 3

## Question 4

In [63]:
cur = collection_youtube.find().sort([("views", -1)]).limit(10)
list(cur)

[{'_id': 'MBdVXkSdhwU',
  'title': "BTS (방탄소년단) 'DNA' Official MV",
  'channel_title': 'ibighit',
  'category_id': 10,
  'tags': 'BIGHIT|빅히트|방탄소년단|BTS|BANGTAN|방탄',
  'views': 41500672,
  'likes': 2010366,
  'dislikes': 78076,
  'comment_total': 736179,
  'thumbnail_link': 'https://i.ytimg.com/vi/MBdVXkSdhwU/default.jpg',
  'date': datetime.datetime(1970, 1, 1, 0, 0),
  'category_name': 'Music'},
 {'_id': 'tt2k8PGm-TI',
  'title': 'ZAYN - Dusk Till Dawn ft. Sia',
  'channel_title': 'ZaynVEVO',
  'category_id': 10,
  'tags': 'Dusk Till Dawn|Pop|RCA Records Label|ZAYN feat. Sia',
  'views': 36323498,
  'likes': 1431683,
  'dislikes': 28049,
  'comment_total': 100661,
  'thumbnail_link': 'https://i.ytimg.com/vi/tt2k8PGm-TI/default.jpg',
  'date': datetime.datetime(1970, 1, 1, 0, 0),
  'category_name': 'Music'},
 {'_id': 'LunHybOKIjU',
  'title': 'Eminem Rips Donald Trump In BET Hip Hop Awards Freestyle Cypher',
  'channel_title': 'BETNetworks',
  'category_id': 24,
  'tags': 'BET|BET Netwo

## Question 5

In [13]:
cur = collection_youtube.aggregate([{"$group" : {"_id" : "$category_name", "averageViewsByCategory" : {"$avg" : "$views"}}}])
list(cur)

[{'_id': 'Pets & Animals', 'averageViewsByCategory': 633906.4210526316},
 {'_id': 'Music', 'averageViewsByCategory': 1442240.943283582},
 {'_id': 'News & Politics', 'averageViewsByCategory': 533575.1262626263},
 {'_id': 'Nonprofits & Activism', 'averageViewsByCategory': 681178.2},
 {'_id': 'Shows', 'averageViewsByCategory': 8726.0},
 {'_id': 'Education', 'averageViewsByCategory': 550585.2659574468},
 {'_id': 'People & Blogs', 'averageViewsByCategory': 892992.28515625},
 {'_id': 'Travel & Events', 'averageViewsByCategory': 426492.22222222225},
 {'_id': 'Entertainment', 'averageViewsByCategory': 1059427.7735849055},
 {'_id': 'Howto & Style', 'averageViewsByCategory': 556002.6679389313},
 {'_id': 'Sports', 'averageViewsByCategory': 618074.711409396},
 {'_id': 'Science & Technology', 'averageViewsByCategory': 861563.6158940397},
 {'_id': 'Gaming', 'averageViewsByCategory': 777213.0357142857},
 {'_id': 'Autos & Vehicles', 'averageViewsByCategory': 622727.8648648649},
 {'_id': 'Comedy', 'ave

## Question 6 

In [27]:
cur = collection_youtube.aggregate([
        {"$group" : {"_id" : "$channel_title", "averageViewsByCategory" : {"$avg" : "$likes"}}},
        {"$sort" : { "averageViewsByCategory" : -1 }}
    ])
list(cur)

[{'_id': 'ibighit', 'averageViewsByCategory': 1533457.0},
 {'_id': 'ZaynVEVO', 'averageViewsByCategory': 1431683.0},
 {'_id': 'BETNetworks', 'averageViewsByCategory': 982842.0},
 {'_id': 'melanie martinez', 'averageViewsByCategory': 921577.0},
 {'_id': 'jypentertainment', 'averageViewsByCategory': 793387.0},
 {'_id': 'Logan Paul Vlogs', 'averageViewsByCategory': 724478.5},
 {'_id': 'Wengie', 'averageViewsByCategory': 656526.0},
 {'_id': 'Alesso', 'averageViewsByCategory': 618436.0},
 {'_id': 'shakiraVEVO', 'averageViewsByCategory': 578266.0},
 {'_id': 'TaylorSwiftVEVO', 'averageViewsByCategory': 508015.0},
 {'_id': 'Ed Sheeran', 'averageViewsByCategory': 461378.0},
 {'_id': 'PewDiePie', 'averageViewsByCategory': 438273.0},
 {'_id': 'jbalvinVEVO', 'averageViewsByCategory': 430513.0},
 {'_id': 'Dude Perfect', 'averageViewsByCategory': 406305.5},
 {'_id': 'Linkin Park', 'averageViewsByCategory': 398487.0},
 {'_id': 'Liza Koshy', 'averageViewsByCategory': 397011.2},
 {'_id': 'Conor Maynard